<a href="https://colab.research.google.com/github/Mohammed-Taasir/IR_2023-GROUP-14/blob/assignment-1/IR_Assignment_1_QUES_1_and_QUES_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp '/content/drive/MyDrive/IR Assignment-1/CSE508_Winter2023_Dataset.zip' '/content/'
!unzip 'CSE508_Winter2023_Dataset.zip' &> /dev/null
!rm 'CSE508_Winter2023_Dataset.zip'

In [16]:
import glob
import pandas as pd
import numpy as np
import string
import os
import re
import pickle

In [5]:
from sortedcontainers import SortedDict, SortedList, SortedSet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import PlaintextCorpusReader 
from nltk.stem.wordnet import WordNetLemmatizer

# * Defining Helper Functions

### Read from files

In [2]:
def getListOfFiles(directory):
    '''
    Parameters:
        directory: type(string)
        
    returns: list of all files in directory with the full path of file
    '''
    
    list_of_files = []
    
    for file_path in os.listdir(directory):
        full_path = os.path.join(directory, file_path)
        if os.path.isfile(full_path):
            list_of_files.append(full_path)
    
    return list_of_files

### Preprocessing Functions

In [3]:
def lowercase(data):
    '''
    Parameters:
        data: type(string)
    
    returns: lowercase of data
    '''
    
    return data.lower()

In [4]:
def perform_word_tokenize(corpus):
    '''
    Parameters:
        corpus: type(string)
    
    returns word-level tokenization of corpus
    '''
    
    return word_tokenize(corpus)

In [5]:
def remove_stopwords_from_tokens(tokens, stopwords_set):
    '''
    Parameters:
        tokens: type(list)
        stopwords_set: type(set)
    
    returns: tokens without stopwords
    '''
    tokens_sans_stopwords = [x for x in tokens if x not in stopwords_set]
    
    return tokens_sans_stopwords

In [6]:
def remove_punctuation_from_tokens(tokens):
    '''
    Parameters:
        tokens: type(list)
    
    returns: tokens without punctuation
    '''
    tokens_sans_punctuation = [x.translate(str.maketrans('', '', string.punctuation)) for x in tokens]
    
    return tokens_sans_punctuation

In [7]:
def remove_blank_space_tokens(tokens):
    '''
    Parameters:
        tokens: type(list)
    
    returns: tokens without blank tokens
    '''
    tokens_sans_blank_space = [x for x in tokens if x!='']
    
    return tokens_sans_blank_space

In [19]:
def preprocess(corpus, stopwords_set, preprocess_type):
    # Convert the text to lower case
    lowercase_corpus = lowercase(corpus)
    #print(len(lowercase_corpus))
    
    # Perform word tokenization (word_tokenize also takes care of whitespace)
    word_tokens = perform_word_tokenize(lowercase_corpus)
    #print(len(word_tokens))
    
    # Remove stopwords from tokens
    word_tokens_sans_stopwords = remove_stopwords_from_tokens(word_tokens, stopwords_set)
    #print(len(word_tokens_sans_stopwords))
    
    # Remove punctuation marks from tokens
    word_tokens_sans_punctuation = remove_punctuation_from_tokens(word_tokens_sans_stopwords)
    #print(len(word_tokens_sans_punctuation))
    
    # Remove blank space tokens
    word_tokens_sans_blank_tokens = remove_blank_space_tokens(word_tokens_sans_punctuation)
    #print(len(word_tokens_sans_blank_tokens))
  
    
    if preprocess_type=='query':
        return word_tokens_sans_blank_tokens    # if its a query return tokens in the same order
    return sorted(list(dict.fromkeys(word_tokens_sans_blank_tokens)))   # if its a whole document then return tokens in sorted manner

In [9]:
def create_file_dictionary(list_of_files):
    '''
    Paramteres:
        list_of_files: type(string)
    
    returns: file_dictionary with integer key and path_of_file as value
    '''
    file_dictionary = {}
    for i in range(len(list_of_files)):
        file_dictionary[i] = list_of_files[i]
    
    return file_dictionary

# Q1. Data Preprocessing

## (i) Relevant Text Extraction

In [6]:
corpus_root='/content/CSE508_Winter2023_Dataset'
corpus=PlaintextCorpusReader(corpus_root,'.*')

In [7]:
filenames=corpus.fileids()
print(filenames)

['cranfield0001', 'cranfield0002', 'cranfield0003', 'cranfield0004', 'cranfield0005', 'cranfield0006', 'cranfield0007', 'cranfield0008', 'cranfield0009', 'cranfield0010', 'cranfield0011', 'cranfield0012', 'cranfield0013', 'cranfield0014', 'cranfield0015', 'cranfield0016', 'cranfield0017', 'cranfield0018', 'cranfield0019', 'cranfield0020', 'cranfield0021', 'cranfield0022', 'cranfield0023', 'cranfield0024', 'cranfield0025', 'cranfield0026', 'cranfield0027', 'cranfield0028', 'cranfield0029', 'cranfield0030', 'cranfield0031', 'cranfield0032', 'cranfield0033', 'cranfield0034', 'cranfield0035', 'cranfield0036', 'cranfield0037', 'cranfield0038', 'cranfield0039', 'cranfield0040', 'cranfield0041', 'cranfield0042', 'cranfield0043', 'cranfield0044', 'cranfield0045', 'cranfield0046', 'cranfield0047', 'cranfield0048', 'cranfield0049', 'cranfield0050', 'cranfield0051', 'cranfield0052', 'cranfield0053', 'cranfield0054', 'cranfield0055', 'cranfield0056', 'cranfield0057', 'cranfield0058', 'cranfield005

In [8]:
print(type(filenames))
print(len(filenames))

<class 'list'>
1400


## First 5 Documents

In [9]:
for i in range(0, 5):
  outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  doc = outfile.read()
  print("------------")
  print("Document: "+str(i+1))
  print("------------")
  print(doc)
  print('-----------------------------------------------------------------')
  print('')
  outfile.close()

------------
Document: 1
------------
<DOC>
<DOCNO>
1
</DOCNO>
<TITLE>
experimental investigation of the aerodynamics of a
wing in a slipstream .
</TITLE>
<AUTHOR>
brenckman,m.
</AUTHOR>
<BIBLIO>
j. ae. scs. 25, 1958, 324.
</BIBLIO>
<TEXT>
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the des

## Extracting content between TITLE and TEXT tag from all documents.

In [10]:
for i in range(len(filenames)):
    outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
    doc = outfile.read()

    # initializing string
    test_str = doc

    # initializing tags
    tag1 = "TITLE"
    tag2 = "TEXT"

    # regex to extract required strings
    reg_str1 = "<"+tag1+">(.*?)</"+tag1+">"
    res1 = re.findall(reg_str1, test_str, re.DOTALL)

    reg_str2 = "<"+tag2+">(.*?)</"+tag2+">"
    res2 = re.findall(reg_str2, test_str, re.DOTALL)

    #Combining contents of TITLE and TEXT
    res = res1+res2
    
    s = res

    # using list comprehension
    listToStr = ' '.join([str(elem) for elem in s])

    writeFile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'w')
    L = listToStr

    writeFile.write(L)
    writeFile.close()

    outfile.close()

## First 5 Documents after extracting contents between TITLE and TEXT tag.

In [11]:
for i in range(0, 5):
  outfile = open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  doc = outfile.read()
  print("------------")
  print("Document: "+str(i+1))
  print("------------")
  print(doc)
  print('-----------------------------------------------------------------')
  print('')
  outfile.close()

------------
Document: 1
------------

experimental investigation of the aerodynamics of a
wing in a slipstream .
 
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the destalling effects was made for
the specific configuration of the experiment .

-----------------------------------------------

# (ii) Preprocessing

## 1. Lowercase the text

In [12]:
newfiles=[]
for i in range(len(filenames)):
  fileind=open('/content/CSE508_Winter2023_Dataset/'+filenames[i], 'r')
  filedata=fileind.read()
  filedata=filedata.lower()
  newfiles.append(filedata)
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------

experimental investigation of the aerodynamics of a
wing in a slipstream .
 
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with supporting
evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or boundary-layer-control
effect .  the integrated remaining lift increment,
after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the destalling effects was made for
the specific configuration of the experiment .

-------------------------------------------------

In [13]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## 2. Perform tokenization

In [14]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  filedata=word_tokenize(filedata)
  newfiles[i] = filedata
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------
['experimental', 'investigation', 'of', 'the', 'aerodynamics', 'of', 'a', 'wing', 'in', 'a', 'slipstream', '.', 'an', 'experimental', 'study', 'of', 'a', 'wing', 'in', 'a', 'propeller', 'slipstream', 'was', 'made', 'in', 'order', 'to', 'determine', 'the', 'spanwise', 'distribution', 'of', 'the', 'lift', 'increase', 'due', 'to', 'slipstream', 'at', 'different', 'angles', 'of', 'attack', 'of', 'the', 'wing', 'and', 'at', 'different', 'free', 'stream', 'to', 'slipstream', 'velocity', 'ratios', '.', 'the', 'results', 'were', 'intended', 'in', 'part', 'as', 'an', 'evaluation', 'basis', 'for', 'different', 'theoretical', 'treatments', 'of', 'this', 'problem', '.', 'the', 'comparative', 'span', 'loading', 'curves', ',', 'together', 'with', 'supporting', 'evidence', ',', 'showed', 'that', 'a', 'substantial', 'part', 'of', 'the', 'lift', 'increment', 'produced', 'by', 'the', 'slipstream', 'was', 'due', 'to', 'a', '/destalling/', 'or', 'boundary-layer-control'

In [15]:
stopwords_set = set(stopwords.words('english'))

## 3. Remove stopwords

In [16]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  tokens_sans_stopwords = [x for x in filedata if x not in stopwords_set]
  newfiles[i] = tokens_sans_stopwords
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------
['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', '.', 'experimental', 'study', 'wing', 'propeller', 'slipstream', 'made', 'order', 'determine', 'spanwise', 'distribution', 'lift', 'increase', 'due', 'slipstream', 'different', 'angles', 'attack', 'wing', 'different', 'free', 'stream', 'slipstream', 'velocity', 'ratios', '.', 'results', 'intended', 'part', 'evaluation', 'basis', 'different', 'theoretical', 'treatments', 'problem', '.', 'comparative', 'span', 'loading', 'curves', ',', 'together', 'supporting', 'evidence', ',', 'showed', 'substantial', 'part', 'lift', 'increment', 'produced', 'slipstream', 'due', '/destalling/', 'boundary-layer-control', 'effect', '.', 'integrated', 'remaining', 'lift', 'increment', ',', 'subtracting', 'destalling', 'lift', ',', 'found', 'agree', 'well', 'potential', 'flow', 'theory', '.', 'empirical', 'evaluation', 'destalling', 'effects', 'made', 'specific', 'configuration', 'experiment', '.']
--

## 4. Remove punctuations

In [17]:
for i in range(len(newfiles)):
  filedata=newfiles[i]
  tokens_sans_punctuation = [x.translate(str.maketrans('', '', string.punctuation)) for x in filedata]
  newfiles[i] = tokens_sans_punctuation
print(len(newfiles))

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
----------
File : 1
----------
['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', '', 'experimental', 'study', 'wing', 'propeller', 'slipstream', 'made', 'order', 'determine', 'spanwise', 'distribution', 'lift', 'increase', 'due', 'slipstream', 'different', 'angles', 'attack', 'wing', 'different', 'free', 'stream', 'slipstream', 'velocity', 'ratios', '', 'results', 'intended', 'part', 'evaluation', 'basis', 'different', 'theoretical', 'treatments', 'problem', '', 'comparative', 'span', 'loading', 'curves', '', 'together', 'supporting', 'evidence', '', 'showed', 'substantial', 'part', 'lift', 'increment', 'produced', 'slipstream', 'due', 'destalling', 'boundarylayercontrol', 'effect', '', 'integrated', 'remaining', 'lift', 'increment', '', 'subtracting', 'destalling', 'lift', '', 'found', 'agree', 'well', 'potential', 'flow', 'theory', '', 'empirical', 'evaluation', 'destalling', 'effects', 'made', 'specific', 'configuration', 'experiment', '']
----------------

## 5. Remove blank space tokens

In [18]:
lent = 0
for i in range(len(newfiles)):
  filedata=newfiles[i]
  tokens_sans_blank_space = [x for x in filedata if x!='']
  newfiles[i] = tokens_sans_blank_space
  lent += len(newfiles[i])
print(len(newfiles))
print(lent)

for i in range(0, 5):
  print('----------')
  print('File : '+str(i+1))
  print('----------')
  print(newfiles[i])
  print('--------------------------------------------------------------------')

1400
127377
----------
File : 1
----------
['experimental', 'investigation', 'aerodynamics', 'wing', 'slipstream', 'experimental', 'study', 'wing', 'propeller', 'slipstream', 'made', 'order', 'determine', 'spanwise', 'distribution', 'lift', 'increase', 'due', 'slipstream', 'different', 'angles', 'attack', 'wing', 'different', 'free', 'stream', 'slipstream', 'velocity', 'ratios', 'results', 'intended', 'part', 'evaluation', 'basis', 'different', 'theoretical', 'treatments', 'problem', 'comparative', 'span', 'loading', 'curves', 'together', 'supporting', 'evidence', 'showed', 'substantial', 'part', 'lift', 'increment', 'produced', 'slipstream', 'due', 'destalling', 'boundarylayercontrol', 'effect', 'integrated', 'remaining', 'lift', 'increment', 'subtracting', 'destalling', 'lift', 'found', 'agree', 'well', 'potential', 'flow', 'theory', 'empirical', 'evaluation', 'destalling', 'effects', 'made', 'specific', 'configuration', 'experiment']
-------------------------------------------------

# QUESTION 2 STARTS HERE


# Q2. Boolean Queries

### 1. Create a unigram inverted index(from scratch; No library allowed) of the dataset obtained from Q1 (after preprocessing).

### 2. Use Python’s pickle module to save and load the unigram inverted index.

In [21]:
def create_unigram_inverted_index(file_dictionary, stopwords_set):
    # initialize unigram inverted index
    unigram_inverted_index = SortedDict()
    
    # unigram inverted index
    for doc_ID in range(len(file_dictionary)):
        file = open(file_dictionary[doc_ID], 'r', encoding='utf-8', errors='ignore')
        file_corpus = file.read()
        file.close()
        doc_tokens = preprocess(file_corpus, stopwords_set, 'doc')
        for token in doc_tokens:
            if token in unigram_inverted_index:
                unigram_inverted_index[token][0] += 1   # stores doc_frequency
                unigram_inverted_index[token][1].add(doc_ID)    # appending docs in which that term occurs
            else:
                unigram_inverted_index[token] = [1, SortedSet([doc_ID])]    # initializing if this token appears for first time
    
    # Storing unigram inverted index
    uii_file = open('unigram_inverted_index_pickle_file', 'wb')
    pickle.dump(unigram_inverted_index, uii_file)
    uii_file.close()

### 3. Provide support for the following operations: 
a. T1 AND T2<br>
b. T1 AND NOT T2<br>
c. T1 OR T2<br>
d. T1 OR NOT T2

AND

In [22]:
def xANDy(pos_list_1, pos_list_2):
    result = SortedSet()
    i,j = 0,0
    num_of_operations = 0
    while(i<len(pos_list_1) and j<len(pos_list_2)):
        if pos_list_1[i]==pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
            j+=1
        elif pos_list_1[i]<pos_list_2[j]:
            i+=1
        else:
            j+=1
        num_of_operations+=1
    
    return result, num_of_operations

OR

In [23]:
def xORy(pos_list_1, pos_list_2):
    result = SortedSet()
    i,j = 0,0
    num_of_operations = 0
    while(i<len(pos_list_1) and j<len(pos_list_2)):
        if pos_list_1[i]==pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
            j+=1
        elif pos_list_1[i]<pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
        else:
            result.add(pos_list_2[j])
            j+=1
        num_of_operations+=1
    
    while(i<len(pos_list_1)):
        result.add(pos_list_1[i])
        i+=1
    while(j<len(pos_list_2)):
        result.add(pos_list_2[j])
        j+=1
    
    return result, num_of_operations

AND NOT

In [24]:
def xANDNOTy(pos_list_1, pos_list_2):
    result = SortedSet()
    i,j = 0,0
    num_of_operations = 0
    while(i<len(pos_list_1) and j<len(pos_list_2)):
        if pos_list_1[i]==pos_list_2[j]:
            i+=1
            j+=1
        elif pos_list_1[i]<pos_list_2[j]:
            result.add(pos_list_1[i])
            i+=1
        else:
            j+=1
        num_of_operations+=1
    
    while(i<len(pos_list_1)):
        result.add(pos_list_1[i])
        i+=1
    
    return result, num_of_operations

OR NOT

In [26]:
def xORNOTy(pos_list_1, pos_list_2, u_list):
    num_of_not_operations = 0
    i,j = 0,0
    not_pos_list_2 = SortedSet()
    while(i<len(pos_list_2) and j<len(u_list)):
        if pos_list_2[i]<u_list[j]:
            i+=1
        elif pos_list_2[i]>u_list[j]:
            not_pos_list_2.add(u_list[j])
            j+=1
        else:
            i+=1
            j+=1
        num_of_not_operations+=1
    while(j<len(u_list)):
        not_pos_list_2.add(u_list[j])
        j+=1
    
    result, ops = xORy(pos_list_1, not_pos_list_2)
    return result, num_of_not_operations+ops

### 4. Queries should be generalized i.e., you should provide support for queries like T1 AND T2 OR T3 OR T4 ....

### 5. You are also required to compute the minimum number of comparisons done to execute the query [only where merging is required]

### 6. Input format: <br>
a. The first line contains N denoting the number of queries to execute<br>
b. The next 2N lines contain queries in the following format:<br>
&ensp; i. Input sequence<br>
&ensp; ii. Operations separated by comma

### 7. Output Format:
a. 4N lines consisting of the results in the following format:<br>
&ensp;i. Query X<br>
&ensp;ii. Number of documents retrieved for query X<br>
&ensp;iii. Names of the documents retrieved for query X<br>
&ensp;iv. Number of comparisons required for query X<br>

### 8. Perform preprocessing steps (from Q1) on the input sequence as well.

In [27]:
def main():
    # create set of stop words for preprocessing
    stopwords_set = set(stopwords.words('english'))
    
    # Get List of Files in Dataset
    list_of_files = getListOfFiles('/content/CSE508_Winter2023_Dataset')
    
    # create dictionary of file with docID (integer) as key and full_path of file as value
    file_dictionary = create_file_dictionary(list_of_files)
    
    # create unigram inverted index once and then load pickle file afterwards
    create_unigram_inverted_index(file_dictionary, stopwords_set)
    
    #Loading pre-processed files
    uii_file = open('unigram_inverted_index_pickle_file', 'rb')
    unigram_inverted_index = pickle.load(uii_file)
    uii_file.close()
    u_list = SortedSet()
    for a in range(len(file_dictionary)):
        u_list.add(a)
    
    N = int(input())
    for q in range(N):
        input_sentence = input()
        input_operation_sequence = input()
        sanitized_query = preprocess(input_sentence, stopwords_set, 'query')
        sanitized_operation_sequence = []
        for op_seq in input_operation_sequence.split(','):
            sanitized_operation_sequence.append(op_seq.lower().strip())
        print(sanitized_query)
        print(sanitized_operation_sequence)
        if(len(sanitized_query)!=len(sanitized_operation_sequence)+1):
            print('invalid query')
            continue    # will continue with next query 
        ptr1,ptr2 = 0,0
        result = unigram_inverted_index[sanitized_query[ptr1]][1]
        num_of_operations = 0
        while(ptr2<len(sanitized_operation_sequence)):
            if sanitized_operation_sequence[ptr2]=='and':
                res, ops = xANDy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1])
            elif sanitized_operation_sequence[ptr2]=='or':
                res, ops = xORy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1])
            elif sanitized_operation_sequence[ptr2]=='and not':
                res, ops = xANDNOTy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1])
            elif sanitized_operation_sequence[ptr2]=='or not':
                res, ops = xORNOTy(result, unigram_inverted_index[sanitized_query[ptr1+1]][1], u_list)
            ptr1+=1
            ptr2+=1
            result = res
            num_of_operations += ops
        print('Number of documents matched: {}'.format(len(result)))
        print('Number of comparisons required: {}'.format(num_of_operations))
        print('Documents: \n')
        for res_doc in result:
            print(file_dictionary[res_doc])

In [ ]:
if __name__ == "__main__":
    main()